In [ ]:
from __future__ import print_function, division

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster")
import os
import glob

import numpy as np

from units import M_solar, m_proton, pc, yr, Myr, km, s, gamma

from injection_helpers import get_SNe

from visualize_helpers import \
    total_radial_momentum_of_snapshot, \
    map_to_all_snapshots, \
    get_snapshot_times, \
    load_snapshots


## 1D (hi-res)

In [ ]:
## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        
from clustered_SNe.analysis.parse import Overview, RunSummary

In [ ]:
data_dir_1D = "1D_data/"

run_summary = RunSummary(data_dir_1D, "25451948-485f-46fe-b87b-f4329d03b203")

# 3D (resolution study)

In [ ]:
base_box_size = 400

N_particles_per_side = np.array([
        150, 
#         200, 
#         250, 
        300,
        600,
    ])
dxs = base_box_size / N_particles_per_side

simulation_base_name = "cluster_cooling"

snapshot_dirs = {N : "../runs/{}_{}/outputs".format(simulation_base_name, N)
                for N in N_particles_per_side}

snapshot_dirs[300] = "../runs/cluster_cooling_small_300/outputs/"

In [ ]:
SNe_dir = "../runs/{}_{}/inputs".format(simulation_base_name, N_particles_per_side[0])
SNe = get_SNe(SNe_dir)
SN_times_3D = np.array([SN.time for SN in SNe])

In [ ]:
def get_times_and_radial_momenta(snapshot_dir):    
    ts = load_snapshots(snapshot_dir)

    times_snapshots_tmp = get_snapshot_times(ts)
    
    #########

    radial_momenta_tmp = map_to_all_snapshots(snapshot_dir, total_radial_momentum_of_snapshot)
        
    return times_snapshots_tmp, radial_momenta_tmp

In [ ]:
from multiprocessing import Pool

pool = Pool(3)

# Read in HD runs

In [ ]:
snapshot_dirs_tmp = [snapshot_dirs[N_particles_per_side_i]
                     for N_particles_per_side_i in N_particles_per_side]

for snapshot_dir in snapshot_dirs_tmp:
    print("Loading {:>3} snapshots from {}".format(
            len(glob.glob(os.path.join(snapshot_dir, "snapshot*.hdf5"))),
            snapshot_dir))

outputs = pool.map(get_times_and_radial_momenta, snapshot_dirs_tmp)

times_snapshots = {N_particles_per_side_i : outputs[i][0]
                   for i, N_particles_per_side_i in enumerate(N_particles_per_side)}

radial_momenta  = {N_particles_per_side_i : outputs[i][1]
                   for i, N_particles_per_side_i in enumerate(N_particles_per_side)}


# Read in MHD runs

In [ ]:
base_box_size = 400

N_particles_per_side_mhd = np.array([
#         150, 
        200, 
#         250, 
#         300,
#         600,
    ])
dxs_mhd = base_box_size / N_particles_per_side_mhd

simulation_base_name_mhd = "cluster_cooling_mhd"

snapshot_dirs_mhd = {N : "../runs/{}_{}/outputs".format(simulation_base_name_mhd, N)
                     for N in N_particles_per_side_mhd}

In [ ]:
snapshot_dirs_tmp = [snapshot_dirs_mhd[N_particles_per_side_i]
                     for N_particles_per_side_i in N_particles_per_side_mhd]

for snapshot_dir in snapshot_dirs_tmp:
    print("Loading {:>3} snapshots from {}".format(
            len(glob.glob(os.path.join(snapshot_dir, "snapshot*.hdf5"))),
            snapshot_dir))

outputs = pool.map(get_times_and_radial_momenta, snapshot_dirs_tmp)

times_snapshots_mhd = {N_particles_per_side_i : outputs[i][0]
                       for i, N_particles_per_side_i in enumerate(N_particles_per_side_mhd)}

radial_momenta_mhd  = {N_particles_per_side_i : outputs[i][1]
                       for i, N_particles_per_side_i in enumerate(N_particles_per_side_mhd)}


# Plot

In [ ]:
with sns.axes_style("ticks"):
    plt.plot((run_summary.times - run_summary.overview.SNe_times[0]) / (Myr),
             run_summary.momentum / (100 * M_solar * 11 * km / s),
             label="1D",
             linestyle="dashed",
             )


    for N_particles_per_side_i in N_particles_per_side[::-1]:

        plt.plot(times_snapshots[N_particles_per_side_i],
                 radial_momenta[N_particles_per_side_i] / (100 * M_solar * 11 * km / s),
                 label="3D: $\Delta x = $ {:.1f} pc".format(400 / N_particles_per_side_i),
                 )
        
    for N_particles_per_side_i in N_particles_per_side_mhd[::-1]:

        plt.plot(times_snapshots_mhd[N_particles_per_side_i],
                 radial_momenta_mhd[N_particles_per_side_i] / (100 * M_solar * 11 * km / s),
                 label="3D MHD: $\Delta x = $ {:.1f} pc".format(400 / N_particles_per_side_i),
                 linestyle="dotted", color="k",
                )


    sns.rugplot(SN_times_3D, color="k", linewidth=3)

    plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
    plt.ylabel(r"$p / (100$ $M_\odot$ $N_\mathrm{SNe})$ $[\mathrm{km}$ $\mathrm{s}^{-1}]$")


    plt.xlim(xmin=-1,
             xmax=100)
    plt.ylim(ymin=300,
             ymax=3000,
            )

    plt.yscale("log")

    plt.legend(loc="best")

In [ ]:
SN_times_3D